In [1]:
import inspect
import logging
import os

import pandas as pd

from csrank.util import setup_logger
from experiments.dbconnection import DBConnector

Using TensorFlow backend.


In [2]:
DIR_PATH = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
log_path = os.path.join(DIR_PATH, 'logs', 'results.log')
setup_logger(log_path=log_path)
logger = logging.getLogger('Result parsing')
config_file_path = os.path.join(DIR_PATH, 'config', 'clusterdb.json')
RESULTS = 'results'
DATASET = "mnist_dc"

In [3]:
self = DBConnector(config_file_path=config_file_path, is_gpu=False, schema='pymc3')

In [4]:
self.fetch_job_arguments(cluster_id=1234)
self.job_description['job_id']

jobs available [1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019]
Job selected : 1002

job_id => 1002
fold_id => 0
dataset => synthetic_dc
learner => nested_logit_model
experiment_schema => results
experiment_table => discrete_choice
dataset_params => {'dataset_type': 'medoid', 'n_test_instances': 100000, 'n_train_instances': 10000, 'n_features': 5, 'n_objects': 10}
fit_params => {'sampler': 'vi', 'sample_params': {'tune': 10, 'draws': 10, 'chains': 1, 'njobs': 1}, 'vi_params': {'n': 30000, 'method': 'advi'}, 'draws': 500}
learner_params => {}
hp_ranges => {'nested_logit_model': {'loss_function': ['categorical_crossentropy', 'binary_crossentropy', 'categorical_hinge'], 'regularization': ['l1', 'l2'], 'n_nests': [2, 5]}}
hp_fit_params => {}
hp_iters => 20
is_gpu => False
seed => 42
inner_folds => 1
duration => 3D
learning_problem => discrete_choice
validation_loss => None
hash_value => a63d1a20a2b93dd5d7955f921aaf2656fe5ab70a
job

1002

In [9]:
self.job_description

{'dataset': 'synthetic_dc',
 'dataset_params': {'dataset_type': 'medoid',
  'n_features': 5,
  'n_objects': 10,
  'n_test_instances': 100000,
  'n_train_instances': 10000},
 'duration': '3D',
 'experiment_schema': 'results',
 'experiment_table': 'discrete_choice',
 'fit_params': {'draws': 500,
  'sample_params': {'chains': 1, 'draws': 10, 'njobs': 1, 'tune': 10},
  'sampler': 'vi',
  'vi_params': {'method': 'advi', 'n': 30000}},
 'fold_id': 0,
 'hash_value': 'f359fe9df00f6bd1cbf62626c9cf37f98fe7a993',
 'hp_fit_params': {},
 'hp_iters': 20,
 'hp_ranges': {'generalized_extreme_value': {'loss_function': ['categorical_crossentropy',
    'binary_crossentropy',
    'categorical_hinge'],
   'n_nests': [3, 6],
   'regularization': ['l1', 'l2']}},
 'inner_folds': 1,
 'is_gpu': False,
 'job_allocated_time': datetime.datetime(2018, 7, 26, 13, 36, 7, 598287),
 'job_id': 1003,
 'learner': 'generalized_extreme_value',
 'learner_params': {},
 'learning_problem': 'discrete_choice',
 'seed': 42,
 'vali

In [6]:
job_desc = self.job_description
job_desc['fold_id'] = 1
job_id = job_desc['job_id']
#del job_desc['job_id']
keys = list(job_desc.keys())
columns = ', '.join(keys)
import numpy as np
print(index)
keys[0] = str(self.job_description['fold_id'])
#keys[-2] = self.job_description["hash_value"]
values_str = ', '.join(keys)
avail_jobs = "{}.avail_jobs".format(self.schema)
running_jobs = "{}.running_jobs".format(self.schema)
insert_result = "INSERT INTO {0} ({1}) SELECT {2} FROM {0} where {0}.job_id = {3} RETURNING job_id".format(avail_jobs, columns, values_str, job_id)
print("Inserting results: {}".format(insert_result))
self.init_connection(cursor_factory=None)
self.cursor_db.execute(insert_result)
job_id = self.cursor_db.fetchone()[0]
insert_job = """INSERT INTO {0} (job_id, cluster_id ,finished, interrupted) 
                                    VALUES ({1}, {2},FALSE, FALSE)""".format(running_jobs, job_id, cluster_id)
self.cursor_db.execute(insert_job)
print(job_id)
print(self.cursor_db.rowcount == 1)
self.close_connection()

6
Inserting results: INSERT INTO pymc3.avail_jobs (dataset, dataset_params, duration, experiment_schema, experiment_table, fit_params, fold_id, hash_value, hp_fit_params, hp_iters, hp_ranges, inner_folds, is_gpu, job_allocated_time, job_id, learner, learner_params, learning_problem, seed, validation_loss) SELECT 1, dataset_params, duration, experiment_schema, experiment_table, fit_params, fold_id, hash_value, hp_fit_params, hp_iters, hp_ranges, inner_folds, is_gpu, job_allocated_time, job_id, learner, learner_params, learning_problem, seed, validation_loss FROM pymc3.avail_jobs where pymc3.avail_jobs.job_id = 1001 RETURNING job_id


IntegrityError: FEHLER:  doppelter Schlüsselwert verletzt Unique-Constraint »avail_jobs_job_id_pk«
DETAIL:  Schlüssel »(job_id)=(1001)« existiert bereits.


In [10]:
discrete_choice_metrics = {'CategoricalAccuracy': 1,
                           'CategoricalTopK2': 2,
                           'CategoricalTopK3': 3,
                           'CategoricalTopK4': 4,
                           'CategoricalTopK5': 5,
                           'CategoricalTopK6': 6}
[x.lower() for x in discrete_choice_metrics.keys()]

['categoricalaccuracy',
 'categoricaltopk2',
 'categoricaltopk3',
 'categoricaltopk4',
 'categoricaltopk5',
 'categoricaltopk6']

In [17]:
self.init_connection()
results_table = "results.discrete_choice"
avail_jobs = "{}.avail_jobs".format(self.schema)
select_st = "SELECT dataset_params, learner, categoricalaccuracy, categoricaltopk2, categoricaltopk3, categoricaltopk5, categoricaltopk6  from {0} INNER JOIN {1} ON {0}.job_id = {1}.job_id where {1}.dataset=\'{2}\'".format(
    results_table, avail_jobs, DATASET)
self.cursor_db.execute(select_st)

In [18]:
data = []
for job in self.cursor_db.fetchall():
    values = list(job.values())
    keys = list(job.keys())
    columns = ['dataset'] +keys[1:]
    vals = [job['dataset_params']['dataset_type'].upper()] + values[1:]
    data.append(vals)

In [19]:
avail_jobs = "{}.avail_jobs".format(self.schema)
select_st = "SELECT dataset_params, learner, categoricalaccuracy, categoricaltopk2, categoricaltopk3, categoricaltopk5, categoricaltopk6  from {0} INNER JOIN {1} ON {0}.job_id = {1}.job_id where {1}.dataset='synthetic_dc'".format(
    results_table, avail_jobs)
self.cursor_db.execute(select_st)

In [20]:
df_full = pd.DataFrame(data, columns=columns)
df_full.head()

,dataset,learner,categoricalaccuracy,categoricaltopk2,categoricaltopk3,categoricaltopk5,categoricaltopk6
0,UNIQUE,ranknet_dc,0.1077,0.2162,0.3267,0.5378,0.6417
1,MEDIAN,ranknet_dc,0.5043,0.6437,0.7272,0.8361,0.8757
2,LARGEST,ranknet_dc,0.8901,0.9400,0.9604,0.9813,0.9872
3,LARGEST,fate_dc,0.4177,0.5799,0.6848,0.8254,0.8753
4,LARGEST,feta_dc,0.8600,0.9244,0.9510,0.9772,0.9842


In [21]:
grouped = df_full.groupby(['dataset', 'learner'])
data = []
for name, group in grouped:
    one_row = [name[0], str(name[1]).upper()]
    std = group.std(axis=0).values
    mean = group.mean(axis=0).values
    one_row.extend(["{:.3f}+-{:.3f}".format(m, s) for m, s in zip(mean, std)])
    data.append(one_row)

In [22]:
df = pd.DataFrame(data, columns=columns)
df.sort_values(by='dataset')
df_path = os.path.join(DIR_PATH, 'results' , DATASET+'.csv')
df.to_csv(df_path)
df

,dataset,learner,categoricalaccuracy,categoricaltopk2,categoricaltopk3,categoricaltopk5,categoricaltopk6
0,LARGEST,FATE_DC,0.418+-nan,0.580+-nan,0.685+-nan,0.825+-nan,0.875+-nan
1,LARGEST,FETA_DC,0.860+-nan,0.924+-nan,0.951+-nan,0.977+-nan,0.984+-nan
2,LARGEST,RANKNET_DC,0.890+-nan,0.940+-nan,0.960+-nan,0.981+-nan,0.987+-nan
3,LARGEST,RANKSVM_DC,0.902+-0.005,0.964+-0.003,0.984+-0.001,0.996+-0.001,nan+-nan
4,MEDIAN,RANKNET_DC,0.504+-nan,0.644+-nan,0.727+-nan,0.836+-nan,0.876+-nan
5,MEDIAN,RANKSVM_DC,0.324+-0.045,0.563+-0.080,0.747+-0.098,0.936+-0.069,nan+-nan
6,UNIQUE,FATE_DC,0.941+-nan,0.972+-nan,0.983+-nan,0.992+-nan,0.995+-nan
7,UNIQUE,FETA_DC,0.229+-nan,0.336+-nan,0.436+-nan,0.624+-nan,0.710+-nan
8,UNIQUE,RANKNET_DC,0.108+-nan,0.216+-nan,0.327+-nan,0.538+-nan,0.642+-nan
9,UNIQUE,RANKSVM_DC,0.146+-0.003,0.246+-0.005,0.349+-0.005,0.547+-0.003,0.878+-0.155


In [39]:
grouped = df.groupby(['dataset'])
for name, group in grouped:
    df_path = os.path.join(DIR_PATH, 'results' , name.lower()+'.csv')
    group.to_csv(df_path)

In [18]:
import numpy as np
np.arange(48,87)

array([48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
       65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
       82, 83, 84, 85, 86])

In [43]:
X_train = np.arange(40).reshape(4,5,2)

In [44]:
learner_params = {}
learner_params['n_objects'], learner_params['n_object_features'] = X_train.shape[1:]

In [45]:
learner_params

{'n_object_features': 2, 'n_objects': 5}